#### TODO

1) initial results are pretty bad. I think we need to not have the previous sale of the house in the dataset. Need to be creative to find the best way to add in the previous house's information into the most recent sale.

2) Did a knn imputer to fill in missing values. This filled in the "prev_sale" column with possibly misleading numbers. Implementing #1 should solve this



### Feature Engineering

In [60]:
import pandas as pd
import numpy as np
df = pd.read_csv('/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v1/data/cleansed_df.csv', index_col=0)

In [61]:
#allow pandas to display all columns
pd.set_option('display.max_columns', 400)

In [62]:
df.shape

(4389, 56)

In [63]:
#get columns with missing value percent less than 12%
ordinal = ['general.propertyTypeDetail'] 

one_hot = ['building.styleCode', 'building.roofCover','building.exteriorWalls', 'building.heatSource', 'hoa_type' ] #'mls.heatingTypes', 'mls.coolingTypes',

small_missing = ['building.fullBathroomCount', 'mls.bathroomCount', 'mls.bedroomCount',
       'mls.lotSizeSquareFeet', 'EvaporativeCooler', 'SolarHeat', 'Fireplace',
       'AC', 'Garage', 'Spa', 'Pool', 'AttachedGarage', 'SpaorHotTub',
       'building.buildingQualityCode', 'mls.floorCount', 'building.heatSource',
       'building.heatSourceCode']



Aggregation filling

In [64]:
# df = df.merge(df.groupby('zip')['sale_price'].mean(), on = 'zip', suffixes= ('','_y'))  #TODO: Caution: Might be an overfit when running with multiple sales. Also, might be worth to only average per zip, per year.
# df.rename(columns = {'sale_price_y': 'average_zip_price'},  inplace= True,  )

#### Create Age Column

In [65]:
df['age'] = df['sale.lastSale.saleDate'] - df['building.yearBuilt']

#### Get Overall quality, average quality, average standard deviation of quality, and average max and min

In [66]:
df['overall_quality'] = df[['Kitchen_level_tagged', 'Bathroom_mean_tagged', 'Floor_mean_tagged', 'Walls_mean_tagged', 'Outview_tagged']].mean(axis=1)
df['average_std_tagged'] = df[['Bathroom_std_tagged', 'Floor_std_tagged', 'Walls_std_tagged', 'Outview_std_tagged', 'Kitchen_level_std_tagged']].mean(axis=1)
df['average_max_tagged'] = df[['Bathroom_max_tagged', 'Floor_max_tagged', 'Walls_max_tagged']].mean(axis=1)
df['average_min_tagged'] = df[['Bathroom_min_tagged', 'Floor_min_tagged', 'Walls_min_tagged']].mean(axis=1)

Index(['address.houseNumber', 'street', 'city', 'state', 'zip',
       'address.zipPlus4', 'sold_month', 'building.fullBathroomCount',
       'lot.lotDepthFeet', 'mls.lotSizeSquareFeet', 'mls.bedroomCount',
       'mls.floorCount', 'mls.bathroomCount', 'building.buildingQualityCode',
       'assessment.totalAssessedValue', 'sale.priorSale.price',
       'bedrooms_tagged', 'living_area_tagged', 'Bathroom_max_tagged',
       'Bathroom_min_tagged', 'Bathroom_std_tagged', 'Bathroom_mean_tagged',
       'Floor_max_tagged', 'Floor_min_tagged', 'Floor_std_tagged',
       'Floor_mean_tagged', 'Walls_max_tagged', 'Walls_min_tagged',
       'Walls_std_tagged', 'Walls_mean_tagged', 'Outview_tagged',
       'Outview_std_tagged', 'Kitchen_level_tagged',
       'Kitchen_level_std_tagged', 'general.propertyTypeDetail',
       'assessment.assessmentYear', 'sale.lastSale.saleDate',
       'sale.priorSale.saleDate', 'building.yearBuilt', 'sale.lastSale.price',
       'building.styleCode', 'building.roof

#### Get Dummies

In [67]:
df = pd.get_dummies(df, columns= one_hot)

#### Ordinal Encoding

In [68]:
from sklearn.preprocessing import OrdinalEncoder
# df.loc[:, ordinal] = ord.fit_transform(df['building.buildingQualityCode'].values.reshape(len(df['building.buildingQualityCode']), 1))
ord_encoders = {}
for col in ordinal:
    ord = OrdinalEncoder(handle_unknown= 'use_encoded_value', unknown_value= len(df[col]) +1)
    ord_encoders[col] = ord
    df[col] = ord.fit_transform(df[col].values.reshape(len(df[col]), 1))

# for col in ordinal:
#     df[col] = ord_encoders[col].transform(df[col])

#### Outlier Removal: Remove all rows where chosen feature has a zscore of less than X amount

In [69]:
from scipy import stats
statcols = ['sale.lastSale.price']
# df['statcol'] = df[target] - df[target]
# statcols = ['statcol']
abs_z_scores = np.abs(stats.zscore(df[statcols]))
filtered_entries = (abs_z_scores < 3.5).all(axis=1)
train = df[filtered_entries]
# train.drop(['statcol'], axis = 1, inplace = True)

In [70]:
print(df.shape, train.shape)

(4389, 86) (4347, 86)


In [71]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in train.columns.values]


In [76]:
train.to_csv('/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v1/data/prepared_data.csv', index = False)